# Formula 1 Driver Greatness Index (DGI) Notebook
This notebook calculates a composite metric, **Driver Greatness Index (DGI)**, to identify the all-time best Formula 1 drivers based on various performance indicators.

## Step 1: Set Up and Connect to the Database

In [ ]:
from sqlalchemy import create_engine
import pandas as pd

# Connect to the SQLite database
db_path = 'sqlite/f1db.sqlite'  # Update to your local path
engine = create_engine(f'sqlite:///{db_path}')

# List available tables
tables = engine.table_names()
print("Available tables:", tables)

## Step 2: Load and Explore Relevant Tables

In [ ]:
# Load tables
drivers = pd.read_sql('SELECT * FROM drivers', engine)
results = pd.read_sql('SELECT * FROM results', engine)
races = pd.read_sql('SELECT * FROM races', engine)
constructors = pd.read_sql('SELECT * FROM constructors', engine)
driver_standings = pd.read_sql('SELECT * FROM driverStandings', engine)

# Preview key data
print(drivers.head())
print(results.head())
print(races.head())
print(constructors.head())
print(driver_standings.head())

## Step 3: Merge and Prepare Data

In [ ]:
# Merge results with races
results_races = results.merge(races, on='raceId', suffixes=('', '_race'))

# Add driver and constructor details
results_races_drivers = results_races.merge(drivers, on='driverId', suffixes=('', '_driver'))
results_full = results_races_drivers.merge(constructors, on='constructorId', suffixes=('', '_constructor'))
print(results_full.head())

## Step 4: Calculate Metrics

In [ ]:
# Teammate dominance
teammate_performance = results_full.groupby(['raceId', 'constructorId']).apply(
    lambda group: group.sort_values('positionOrder')
).reset_index(drop=True)
teammate_performance['is_teammate_beaten'] = (
    teammate_performance.groupby(['raceId', 'constructorId'])['positionOrder']
    .rank(method='min', ascending=True) == 1
)
teammate_dominance = teammate_performance.groupby('driverId')['is_teammate_beaten'].mean() * 100

# Podium percentage
podium_finishes = results_full[results_full['positionOrder'] <= 3]
podium_percentage = podium_finishes.groupby('driverId').size() / results_full.groupby('driverId').size() * 100
podium_percentage.fillna(0, inplace=True)

# Wins from non-pole
non_pole_wins = results_full[(results_full['positionOrder'] == 1) & (results_full['grid'] > 1)]
non_pole_points = non_pole_wins.groupby('driverId')['grid'].sum()

# Pole positions
pole_positions = results_full[results_full['grid'] == 1].groupby('driverId').size()

# Championship wins
championship_wins = driver_standings[driver_standings['position'] == 1].groupby('driverId').size()
championship_wins.fillna(0, inplace=True)

# Longevity and versatility
career_span = results_full.groupby('driverId')['year'].agg(['min', 'max'])
career_length = career_span['max'] - career_span['min'] + 1
num_constructors = results_full.groupby('driverId')['constructorId'].nunique()

print(teammate_dominance.head())
print(podium_percentage.head())

## Step 5: Combine Metrics into the Driver Greatness Index (DGI)

In [ ]:
# Combine metrics into a DataFrame
metrics = pd.DataFrame({
    'teammate_dominance': teammate_dominance,
    'podium_percentage': podium_percentage,
    'non_pole_points': non_pole_points,
    'pole_positions': pole_positions,
    'championship_wins': championship_wins,
    'career_length': career_length,
    'num_constructors': num_constructors,
}).fillna(0)

# Normalize metrics
metrics_normalized = (metrics - metrics.min()) / (metrics.max() - metrics.min())

# Assign weights and calculate DGI
weights = {
    'teammate_dominance': 0.25,
    'podium_percentage': 0.20,
    'non_pole_points': 0.20,
    'pole_positions': 0.15,
    'championship_wins': 0.10,
    'career_length': 0.05,
    'num_constructors': 0.05,
}
metrics_normalized['DGI'] = sum(metrics_normalized[col] * weight for col, weight in weights.items())

# Sort by DGI
metrics_normalized = metrics_normalized.sort_values('DGI', ascending=False)
print(metrics_normalized.head(10))

## Step 6: Visualize the Top 30 Drivers by DGI

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Prepare data for visualization
top_30_drivers = metrics_normalized.nlargest(30, 'DGI').reset_index()
top_30_drivers = top_30_drivers.merge(drivers[['driverId', 'forename', 'surname']], on='driverId')
top_30_drivers['full_name'] = top_30_drivers['forename'] + " " + top_30_drivers['surname']

# Sort for visualization
top_30_drivers = top_30_drivers.sort_values('DGI', ascending=True)

# Plot
plt.figure(figsize=(12, 10))
sns.barplot(
    x='DGI',
    y='full_name',
    data=top_30_drivers,
    palette='viridis'
)
plt.title('Top 30 Formula 1 Drivers by Driver Greatness Index (DGI)', fontsize=16)
plt.xlabel('Driver Greatness Index (DGI)', fontsize=12)
plt.ylabel('Driver', fontsize=12)
plt.tight_layout()
plt.show()